In [1]:
# # # Parameters
DT = "2024-06-11"   

## **One Model Profile 생성 작업**

- 개요
    - One Model Profile 생성 작업
- 내용
    - 최근 구독한(7일내) 단기 preference 정보 저장(item 기반)

In [2]:
from skt.gcp import get_bigquery_client, bq_insert_overwrite, get_max_part, bq_to_df, bq_to_pandas, pandas_to_bq_table, load_query_result_to_table, df_to_bq_table
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F
from datetime import datetime, date, timedelta
from skt.ye import get_spark
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 프로파일 : subsc

In [3]:
# DT = "2024-06-02"
current_date = datetime.strptime(DT, "%Y-%m-%d")
DT_threshold1 = (current_date - timedelta(days=60)).strftime("%Y-%m-%d")
DT_threshold2 = (current_date - timedelta(days=7)).strftime("%Y-%m-%d")

print("DT : ", DT)
print("DT_threshold : ", DT_threshold1)
print("DT_threshold : ", DT_threshold2)

DT :  2024-06-11
DT_threshold :  2024-04-12
DT_threshold :  2024-06-04


In [4]:
query = f"""
    select *
    from adot_reco.recgpt_log_sequence_subsc_v3_daily_prd
    where dt >= '{DT_threshold1}' 
"""

In [5]:
data = bq_to_df(query)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 15:42:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/06/11 15:43:19 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [7]:
# data.count()

In [9]:
# data.select("type").distinct().show()

In [10]:
data.show(n=10)

+--------------------+-------------------+----------+--------+----+--------------------+--------+----+----+----------+--------+
|        svc_mgmt_num|               item|      cat1|    cat2|cat3|           unix_time|second_s|etc1|etc2|        dt|    type|
+--------------------+-------------------+----------+--------+----+--------------------+--------+----+----+----------+--------+
|3f9b1f55a35e69480...|11번가/ Amazon 혜택|NM00000027|옵션Only|해지|1712275310.000000000|     110|null|null|2024-04-05|subsc_v3|
|f3ce03c59993490cb...|11번가/ Amazon 혜택|NM00000027|옵션Only|해지|1712277125.000000000|    1925|null|null|2024-04-05|subsc_v3|
|b2d0100592fefbeb2...|11번가/ Amazon 혜택|NM00000027|옵션Only|가입|1712295269.000000000|   20069|null|null|2024-04-05|subsc_v3|
|e3b0c44298fc1c149...|11번가/ Amazon 혜택|NM00000027|옵션Only|가입|1712309901.000000000|   34701|null|null|2024-04-05|subsc_v3|
|df356429bf525026a...|11번가/ Amazon 혜택|NM00000027|옵션Only|해지|1712244199.000000000|   55399|null|null|2024-04-05|subsc_v3|
|8ed19f4deac42b6

In [15]:
data.select('cat3').distinct().show()

In [11]:
data.select('cat2').distinct().show()

+--------------+
|          cat2|
+--------------+
|      옵션Only|
|     단품/옵션|
|패키지기본혜택|
|        패키지|
|      단품Only|
+--------------+



In [12]:
data.select('cat1').distinct().show()

+----------+
|      cat1|
+----------+
|NM00000304|
|NM00000286|
|NM00000146|
|NM00000259|
|NM00000104|
|NM00000085|
|NM00000036|
|NM00000155|
|NM00000258|
|NM00000330|
|NM00000295|
|NM00000160|
|NM00000308|
|NM00000353|
|NM00000274|
|NM00000227|
|NM00000027|
|NM00000094|
|NM00000065|
|NM00000311|
+----------+
only showing top 20 rows



In [13]:
data.select('item').distinct().show()

+-----------------------+
|                   item|
+-----------------------+
|        배달의민족 구독|
|   우주패스 패밀리 멤버|
|   가벼운샘 500mL x 4팩|
|        FRITZ 원두 세트|
|         독서평설첫걸음|
|   얼라이브 공식몰 구독|
|        진맛과 간편과일|
|           오뚜기밥현미|
|          굽네치킨 구독|
|          우주패스 핫픽|
|           초등독서평설|
|      가벼운샘 2L x 2팩|
|            11pay point|
|      가벼운샘 2L x 4팩|
|             Google_One|
|          SK스토아 구독|
|탄탄한 비건 콜라겐 앰플|
|      폴 바셋 4900 구독|
|       Google One_100GB|
|            B tv_Y 구독|
+-----------------------+
only showing top 20 rows



In [6]:
def prefer_list(data, cate="cat1", thre=0.7):
    data = data.select(['svc_mgmt_num',f'{cate}']).na.drop("any", subset=f'{cate}')
    data = data.groupby('svc_mgmt_num',f'{cate}').agg(count("*").alias(f'{cate}_cnt'))
    data_user = data.groupby('svc_mgmt_num').agg(max(f'{cate}_cnt').alias(f'{cate}_max_cnt'))
    data = data.join(data_user, on='svc_mgmt_num', how="left")
    data = data.withColumn(f'{cate}_prop', col(f'{cate}_cnt')/col(f'{cate}_max_cnt'))
    data = data.filter(col(f'{cate}_prop') > thre)
    
    window_spec = Window.partitionBy("svc_mgmt_num").orderBy(desc(f'{cate}_prop'))

    top_keywords = data.withColumn("rank", row_number().over(window_spec)) \
                    .filter(col("rank") <= 10) \
                    .drop("rank")
    
    # data = top_keywords.groupBy("svc_mgmt_num").agg(collect_list(f'{cate}').alias(f'{cate}_list'))
    data = top_keywords.groupBy("svc_mgmt_num").agg(concat_ws(", ",collect_list(f'{cate}')).alias(f'{cate}_list'))

    
    return data

## subsc item level

In [7]:
data_item = data.filter(col('dt') >= f'{DT_threshold2}')
data_item = data_item.select(['svc_mgmt_num','item']).na.drop("any", subset='item')

In [8]:
data_item1 = prefer_list(data_item, cate="item", thre=0.5)

### 전체 데이터 Merge

In [9]:
full_data = data_item1.withColumn("dt", lit(DT).cast("date"))

In [21]:
full_data.show(n=10)

+--------------------+--------------------------------+----------+
|        svc_mgmt_num|                       item_list|        dt|
+--------------------+--------------------------------+----------+
|0009ae7829db495e9...|     [배달의민족 구독, Google...|2024-06-04|
|000f1de87158e5b82...|   [우주패스 life, 투썸플레이...|2024-06-04|
|00123039dc03efd62...|      [Wavve 앤 데이터 구독, ...|2024-06-04|
|00140e3c42a29ace4...|[배스킨라빈스 구독, 우주패스 ...|2024-06-04|
|001672e82b49911b5...|        [11번가/ Amazon 혜택,...|2024-06-04|
|00168c950dfcfb771...|[배달의민족 구독, 세븐일레븐 ...|2024-06-04|
|00181456b77b296fe...|            [Google One 100GB...|2024-06-04|
|0019c5b84e65f5d13...| [세븐일레븐 할인, 우주패스 l...|2024-06-04|
|001e10bb04131d233...|              [메가MGC커피 4900]|2024-06-04|
|002385b8cf5f0f93f...|            [Google One 100GB...|2024-06-04|
+--------------------+--------------------------------+----------+
only showing top 10 rows



In [10]:
full_data = full_data.withColumnRenamed("item_list", "subsc_item_prefer")

In [11]:
full_data.printSchema()

root
 |-- svc_mgmt_num: string (nullable = true)
 |-- subsc_item_prefer: string (nullable = false)
 |-- dt: date (nullable = true)



### AIDP 저장

In [12]:
dest_dataset = "x1113099"
partitioned_dest_table = "one_model_profile_subsc"

In [13]:
get_bigquery_client().query(f"""
    CREATE TABLE IF NOT EXISTS {dest_dataset}.{partitioned_dest_table}
    (
        svc_mgmt_num STRING,
        subsc_item_prefer STRING,
        dt DATE,
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : x1113099.one_model_profile_subsc


In [14]:
df_to_bq_table(df=full_data,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")